In [ ]:
import numpy as np
from scipy.integrate import dblquad
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy.io import savemat

### Model

Input: Image $I(x,y)$

Signal: $s(x,y)=f_{DoG}*I(x,y)$

Noise: Multivariate normal distribution with covariance matrix
$\Sigma_{ij}=\sigma_{noise}^2 (\alpha \delta_{ij}+\beta e^{-\Delta_{ij}/2\tau})$ and mean 0

Ouput of single trial:

$b_j=0$ for $s(x_j,y_j)+n_j+offset < 0.5$

$b_j=1$ for $s(x_j,y_j)+n_j+offset > 0.5$

### Functions

In [ ]:
def dog_filter(sig_cen,ratio,weight,on,N_x,N_y):
    sig_sur=sig_cen*ratio
    dog=lambda x,y: on/(2*np.pi*sig_cen**2)*np.exp(-(x**2+y**2)/(2*sig_cen**2))\
    -on*weight/(2*np.pi*sig_sur**2)*np.exp(-(x**2+y**2)/(2*sig_sur**2))
    
    arr=[]
    for i in range (N_y):
        new=[]
        for j in range (N_x):
            new.append(dog(-0.5*N_x+(j+0.5),-0.5*N_y+(i+0.5)))
        arr.append(new)
    arr=np.array(arr)
    return arr

def generate_output(rf,mask_small,I,f_dog_small,f_dog_large,Sigma,trials,offset):
    num_stim,N_x,N_y=I.shape    
    s_small=np.empty((num_stim,N_x,N_y))
    s_large=np.empty((num_stim,N_x,N_y))
    for i in range(num_stim):
        s_small[i,:,:]=fftconvolve(I[i,:,:],f_dog_small[133:267,133:267],mode='same')
        s_large[i,:,:]=fftconvolve(I[i,:,:],f_dog_large[75:325,75:325],mode='same')
    rf=rf.astype(int)
    num_cells=rf.shape[1]
    b=np.zeros((num_stim,trials,num_cells))
    p=np.zeros((num_stim,trials,num_cells))
    p_im=np.zeros((num_stim,trials,num_cells))
    p_rnd=np.zeros((num_stim,trials,num_cells))
    n=np.random.multivariate_normal(np.zeros(num_cells),Sigma,(num_stim,trials))
    for j in range(num_cells):
        if mask_small[j]:
            b[:,:,j]=np.rint(np.clip(np.outer(s_small[:,rf[0,j]+N_x//2,rf[1,j]+N_y//2],np.ones(trials))\
                                     +n[:,:,j]+offset,0,1))
        else:
            b[:,:,j]=np.rint(np.clip(np.outer(s_large[:,rf[0,j]+N_x//2,rf[1,j]+N_y//2],np.ones(trials))\
                                     +n[:,:,j]+offset,0,1))
    return b

### Setting parameters 
(only necessary if original model data is not loaded)

In [ ]:
px_length=2 #pixel size in µm

#DoG filter parameters
center_surround_ratio_small=2.0
center_surround_ratio_large=2.0
weight_surround=0.5
on=1 #cell type (1:on, -1:off)

#Noise
noise_std=0.022 #standard deviation of white noise
alpha=0.45 #independent noise fraction
beta=np.sqrt(1-alpha**2) #correlated noise fraction
tau=30.0  #spatial decay of noise correlations

offset=0.168

N_x=N_y=400 #stimulus size, size of the natural textures

### Generating cell positions
(only necessary if original model data is not loaded, does not generate exactly the same set of cells)

In [ ]:
retinainfo=loadmat('retinainfo.mat')

#getting RF positions, centered around 0
rf_positions=np.array([retinainfo['pos_x'].flatten()-50,retinainfo['pos_y'].flatten()])
sigma_small=retinainfo['rf_size_small'].item()/2 #Factor of 0.5 since given values seem to be overly large
sigma_large=retinainfo['rf_size_large'].item()/2 #compared to the overall area


#randomly masking and removing 40% of the cells to correct for amacrine cells in the sample
choice=np.array([np.random.random(rf_positions.shape[1])>0.6])
choice=np.concatenate((choice,choice))
cells=np.ma.compress_cols(np.ma.masked_array(rf_positions,choice))
#masking two thirds of the cells that should have small receptive fields
mask_small=np.array([np.random.random(cells.shape[1])>0.67]).flatten()

sigma_small=sigma_small/px_length
sigma_large=sigma_large/px_length
cells=cells/px_length
num_cells=cells.shape[1]

### Save or load model data

In [ ]:
# savemat('Model_data.mat',{'cell_positions':cells,'sigma_small_RF':sigma_small,'sigma_large_RF':sigma_large,\
#                           'small_RF_mask':mask_small,'center_surround_ratio_small':center_surround_ratio_small,\
#                           'center_surround_ratio_large':center_surround_ratio_large,\
#                           'weight_surround':weight_surround,'white_noise_std':noise_std,\
#                           'independent_noise_fraction':alpha,'correlated_noise_fraction':beta,\
#                           'noise_spatial_decay':tau,'offset':offset,'pixel_size':px_length})

In [ ]:
loaddict=loadmat('Model_data.mat')

cells=np.array(loaddict['cell_positions'])
sigma_small=loaddict['sigma_small_RF'].item()
sigma_large=loaddict['sigma_large_RF'].item()
mask_small=np.array(loaddict['small_RF_mask']).flatten()
center_surround_ratio_small=loaddict['center_surround_ratio_small'].item()
center_surround_ratio_large=loaddict['center_surround_ratio_large'].item()
weight_surround=loaddict['weight_surround'].item()
noise_std=loaddict['white_noise_std'].item()
alpha=loaddict['independent_noise_fraction'].item()
beta=loaddict['correlated_noise_fraction'].item()
tau=loaddict['noise_spatial_decay'].item()
offset=loaddict['offset'].item()
px_length=loaddict['pixel_size'].item()

N_x=N_y=400 #stimulus size, size of the natural textures
on=1
num_cells=cells.shape[1]

### Generation of filters and noise covariance

In [ ]:
f_dog_small=dog_filter(sigma_small,center_surround_ratio_small,weight_surround,on,400,400)
f_dog_large=dog_filter(sigma_large,center_surround_ratio_large,weight_surround,on,400,400)
Delta=np.array([[np.linalg.norm(cells[:,i]*px_length-cells[:,j]*px_length)\
                 for i in range(num_cells)]for j in range(num_cells)])
Sigma=noise_std**2 *(beta*np.exp(-Delta/(2*tau))+alpha*np.identity(num_cells))

### Natural image stimulus

####  Loading stimulus images

In [ ]:
textures_400=loadmat('textures_400.mat')['textures']
textures=[]
for i in range(textures_400.shape[1]):
    textures.append(textures_400[0,i][0])
textures=np.array(textures)/255.
textures=np.delete(textures,64,0)# excluding two textures with outlying mean 
textures=np.delete(textures,38,0)
num_text=textures.shape[0]

#### Output generation

In [ ]:
trials=300
output=generate_output(cells,mask_small,textures,f_dog_small,f_dog_large,Sigma,trials,offset)

#### Saving output data

In [ ]:
savemat('Retina_model_data_images.mat',{'cell_positions':cells,'sigma_small_RF':sigma_small,\
                                        'sigma_large_RF':sigma_large,'small_RF_mask':mask_small,\
                                        'center_surround_ratio_small':center_surround_ratio_small,\
                                        'center_surround_ratio_large':center_surround_ratio_large,\
                                        'weight_surround':weight_surround,'white_noise_std':noise_std,\
                                        'independent_noise_fraction':alpha,'correlated_noise_fraction':beta,\
                                        'noise_spatial_decay':tau,'offset':offset,'trials':trials,\
                                        'pixel_size':px_length,'output':output})

### Checkerboard stimulus

#### Stimulus generation

In [ ]:
px_size=5 #size of checkerboard pixels
cb_low=0.15 #brightness of dark pixels
cb_high=0.87 #brightness of bright pixels
num_text=2000 #number of checkerboard stimulus images
trials=10 #number of trials per image

px_num=N_x//px_size
checkerboard=np.ones((num_text,N_y,N_x))*cb_low
for i in range (px_num):
    for j in range (px_num):
        checkerboard[:,i*px_size:(i+1)*px_size,j*px_size:(j+1)*px_size]\
        +=np.outer(np.random.binomial(1,0.5,num_text),np.ones((px_size,px_size)))\
        .reshape(num_text,px_size,px_size)*(cb_high-cb_low)

#### Output generation

In [ ]:
output=generate_output(cells,mask_small,checkerboard,f_dog_small,f_dog_large,Sigma,trials,offset)

#### Saving output data

In [ ]:
savemat('Retina_model_data_checkerboard.mat',{'cell_positions':cells,'sigma_small_RF':sigma_small,\
                                              'sigma_large_RF':sigma_large,'small_RF_mask':mask_small,\
                                              'center_surround_ratio_small':center_surround_ratio_small,\
                                              'center_surround_ratio_large':center_surround_ratio_large,\
                                              'weight_surround':weight_surround,'white_noise_std':noise_std,\
                                              'independent_noise_fraction':alpha,'correlated_noise_fraction':beta,\
                                              'noise_spatial_decay':tau,'offset':offset,'trials':trials,\
                                              'pixel_size':px_length,'output':output})

### Fullfield flicker

#### Stimulus generation

In [ ]:
num_text=2000 #number of checkerboard stimulus images
trials=10 #number of trials per image
ff_variance=0.06 #variance of gaussian full field flicker

flicker=np.outer(np.random.normal(0.5,ff_variance,num_text),np.ones((N_x,N_y)))
flicker=flicker.reshape(num_text,N_x,N_y)

#### Output generation

In [ ]:
output=generate_output(cells,mask_small,flicker,f_dog_small,f_dog_large,Sigma,trials,offset)

####  Saving output data

In [ ]:
savemat('Retina_model_data_flicker.mat',{'cell_positions':cells,'sigma_small_RF':sigma_small,\
                                         'sigma_large_RF':sigma_large,'small_RF_mask':mask_small,\
                                         'center_surround_ratio_small':center_surround_ratio_small,\
                                         'center_surround_ratio_large':center_surround_ratio_large,\
                                         'weight_surround':weight_surround,'white_noise_std':noise_std,\
                                         'independent_noise_fraction':alpha,'correlated_noise_fraction':beta,\
                                         'noise_spatial_decay':tau,'offset':offset,'trials':trials,\
                                         'pixel_size':px_length,'output':output})